# Virtualizer + Icechunk + Cubed

In [1]:
import cubed
# import icechunk
import xarray as xr
import virtualizarr
# from icechunk import IcechunkStore, StorageConfig, StoreConfig, VirtualRefConfig

In [2]:
! rm -rf combined rechunked

### Open tutorial data

In [3]:
ds = xr.tutorial.open_dataset('air_temperature')

### Split into two NetCDF files

In [4]:
ds1 = ds.isel(time=slice(None, 1460))
ds2 = ds.isel(time=slice(1460, None))
ds1.to_netcdf('air1.nc')
ds2.to_netcdf('air2.nc')

/var/folders/8t/gslp67x10vgfjgv68qj92n0m0000gn/T/ipykernel_99771/2127577491.py:3: SerializationWarning: saving variable air with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds1.to_netcdf('air1.nc')
/var/folders/8t/gslp67x10vgfjgv68qj92n0m0000gn/T/ipykernel_99771/2127577491.py:4: SerializationWarning: saving variable air with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds2.to_netcdf('air2.nc')


### Combine two NetCDFs using Virtualizer

In [5]:
vds1 = virtualizarr.open_virtual_dataset('air1.nc', indexes={})
vds2 = virtualizarr.open_virtual_dataset('air2.nc', indexes={})
combined_vds = xr.concat([vds1, vds2], dim='time', coords='minimal', compat='override')

### Write references to Icechunk

In [10]:
storage = StorageConfig.filesystem('combined')
store = IcechunkStore.create(storage=storage, mode="w")
combined_vds.virtualize.to_icechunk(store)

### Commit to Icechunk

In [11]:
store.commit("combined files")

'TKCY5X6PF0D8QMGAKRWG'

### Open in Xarray using Cubed arrays

In [14]:
ds = xr.open_zarr(
    store,
    zarr_version=3,
    consolidated=False,
    chunked_array_type='cubed',
    from_array_kwargs={'spec': cubed.Spec(allowed_mem="2GB", executor_name="single-threaded")},
    chunks={}
)
ds

/var/folders/8t/gslp67x10vgfjgv68qj92n0m0000gn/T/ipykernel_98339/3547589359.py:1: FutureWarning: zarr_version is deprecated, use zarr_format
  ds = xr.open_zarr(


<xarray.Dataset> Size: 31MB
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
Data variables:
    air      (time, lat, lon) float64 31MB cubed.Array<chunksize=(730, 13, 27)>
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

### TODO

In [15]:
cubed.to_zarr(ds["air"].data, "rechunked")

ValueError: cannot reshape array of size 1934500 into shape (730,13,27)

In [9]:
ds.to_zarr("rechunked", safe_chunks=False)

/Users/tom/miniforge3/envs/cubed-virtualizarr-example/lib/python3.11/site-packages/xarray/core/dataset.py:2621: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  return to_zarr(  # type: ignore[call-overload,misc]


ValueError: cannot reshape array of size 1934500 into shape (730,13,27)

In [6]:
# Write to Kerchunk format
combined_vds.virtualize.to_kerchunk('combined.json', format='json')

In [7]:
vds = virtualizarr.open_virtual_dataset('combined.json', filetype='kerchunk', indexes={})
vds

<xarray.Dataset> Size: 8MB
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
    lon      (lon) float32 212B ManifestArray<shape=(53,), dtype=float32, chu...
    lat      (lat) float32 100B ManifestArray<shape=(25,), dtype=float32, chu...
    time     (time) float32 12kB ManifestArray<shape=(2920,), dtype=float32, ...
Data variables:
    air      (time, lat, lon) int16 8MB ManifestArray<shape=(2920, 25, 53), d...
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

In [15]:
spec = cubed.Spec(allowed_mem="2GB", executor_name="single-threaded")
ds = xr.open_dataset(
    'combined.json',
    engine="kerchunk",
    # zarr_version=3,
    # consolidated=False,
    chunked_array_type='cubed',
    from_array_kwargs={'spec': spec},
    chunks={}
)
ds

<xarray.Dataset> Size: 31MB
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB cubed.Array<chunksize=(1460, 25, 53)>
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

In [19]:
cubed.to_zarr(ds["air"].data, "rechunked")

In [18]:
ds.to_zarr("rechunked", safe_chunks=False, chunkmanager_store_kwargs={'spec': spec})

ValueError: Arrays must have same spec in single computation. Specs: [cubed.Spec(work_dir=None, allowed_mem=2000000000, reserved_mem=0, executor=<cubed.runtime.executors.local.SingleThreadedExecutor object at 0x1285b8990>, storage_options=None, zarr_compressor=default), cubed.Spec(work_dir=None, allowed_mem=2000000000, reserved_mem=0, executor=<cubed.runtime.executors.local.SingleThreadedExecutor object at 0x1285b8990>, storage_options=None, zarr_compressor=default), cubed.Spec(work_dir=None, allowed_mem=2000000000, reserved_mem=100000000, executor=None, storage_options=None, zarr_compressor=default)]